## Import libraries
In this section the packages are imported.

In [40]:
import pytz
import json
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import ta

import dash
from dash import dcc, html
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import datetime
import math
import matplotlib.pyplot as plt

## Extracting data
In this section the necessary data is extracted. 
The Fear and Greed Index (FGI) is extracted from the CNN website.
The SPY is extracted with yfinance.

In [41]:
START_DATE = '2021-01-04'

In [42]:
def fetch_fear_greed_index(): 
        # in this section the FGI is extracted from the CNN website
        # simulating to be a human
        headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                        "AppleWebKit/537.36 (KHTML, like Gecko) "
                        "Chrome/114.0.0.0 Safari/537.36"
        }

        # the website of CNN and start date of extraction
        BASE_URL = 'https://production.dataviz.cnn.io/index/fearandgreed/graphdata'

        # importing the data from the website
        r = requests.get('{}/{}'.format(BASE_URL, START_DATE),headers=headers)
        data = r.json()

        # the data is converted into a dataframe
        fg_data = data['fear_and_greed_historical']['data']
        fear_greed_values = {}

        # for all the data the date is transformed into a normal datetime
        for data in fg_data:
                dt = datetime.datetime.fromtimestamp(data['x'] / 1000, tz=pytz.utc)
                fear_greed_values[dt.date()] = int(math.ceil(data['y']))

        # the values are put into a dataframe
        fear_greed_values = pd.DataFrame(fear_greed_values.values(),index=fear_greed_values.keys(),columns = ['FGI'])
        fear_greed_values.index = pd.to_datetime(fear_greed_values.index)

        return fear_greed_values

In [43]:
def fetch_data():
    # SPY data
    spy = yf.download('SPY', start=START_DATE, interval='1wk',multi_level_index=False)
    spy.dropna(inplace=True)

    # Technical Indicators
    spy['200DMA'] = spy['Close'].rolling(window=28).mean()
    spy['200DMA perc'] = ((spy['Close'] - spy['200DMA'])/spy['200DMA']) * 100
    spy['50DMA'] = spy['Close'].rolling(window=7).mean()

    spy['RSI'] = ta.momentum.RSIIndicator((spy['Close'].squeeze()), window=14).rsi()
    spy['RSI_Smoothed'] = spy['RSI'].ewm(span=17, adjust=False).mean()

    macd = ta.trend.MACD(spy['Close'].squeeze())
    spy['MACD'] = macd.macd()
    spy['MACD_signal'] = macd.macd_signal()
    spy['MACD_hist'] = macd.macd_diff()
    
    return spy

def fetch_vix():
    vix = yf.download('^VIX', start=START_DATE, interval='1d')
    return vix['Close']

In [ ]:
def computing_signals():
    
    # fetching and concatenating all data, in indicator_total all signals are saved
    spy = fetch_data()
    fgi = fetch_fear_greed_index()
    vix = fetch_vix()
    indicator_total = pd.concat([spy, fgi, vix],axis=1)

    # a new DataFrame is made to save all the buy and sell signals
    buy_sell_signals = pd.DataFrame()

    # a new parameter is developed for the buy signal
    buy_sell_signals['buy'] = ((indicator_total['FGI']*15) + (indicator_total['RSI']*5) + (25*indicator_total['200DMA perc']))/indicator_total['Close']
    
    # calculating local high in last 70 days (=2.5 months) & calculating decrease
    buy_sell_signals['local high'] = buy_sell_signals['buy'].rolling(window=70, min_periods=1).max()
    buy_sell_signals['buy signal decrease'] = ((buy_sell_signals['buy'] - buy_sell_signals['local high']) / buy_sell_signals['local high']) * 100

    # the cutoffs for a light or strong buy signal
    buy_sell_signals['buy light'] = ((buy_sell_signals['buy signal decrease']<-70) & (buy_sell_signals['buy signal decrease']>-80)).astype(int)
    buy_sell_signals['buy strong'] = (buy_sell_signals['buy signal decrease']<-80).astype(int)*2
    buy_sell_signals['close'] = indicator_total['Close']
    # buy_sell_signals = buy_sell_signals.dropna()
    # buy signal can be visualized using this function
    
    # computing sell signals: if the smoother RSI below 65
    df_rsi = indicator_total[['RSI_Smoothed','RSI','Close']].dropna()
    arr = np.where((df_rsi['RSI_Smoothed']<65),1.0,0.0)
    diff_arr = np.insert(np.diff(arr), 0, np.nan)
    df_rsi['sell'] = diff_arr
    buy_sell_signals[['RSI_smoothed','sell']] = df_rsi[['RSI_Smoothed','sell']]
    
    return indicator_total, buy_sell_signals

indicator_total, buy_sell_signals = computing_signals()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [49]:
buy_sell_signals

,RSI_smoothed,sell
2021-04-05,70.313659,NaN
2021-04-12,70.595487,0.0
2021-04-19,70.779438,0.0
2021-04-26,70.972080,0.0
2021-05-03,71.389248,0.0
...,...,...
2025-05-05,48.800534,0.0
2025-05-12,49.769859,0.0
2025-05-19,50.138082,0.0
2025-05-26,50.744796,0.0


In [63]:
buy_sell_signals.iloc[-20:-1]

,buy,local high,buy signal decrease,buy light,buy strong,close,RSI_smoothed,sell
2025-05-12,2.285071,2.285071,0.000000,0,0,594.200012,49.769858,0.0
2025-05-13,NaN,2.285071,NaN,0,0,NaN,NaN,NaN
2025-05-14,NaN,2.285071,NaN,0,0,NaN,NaN,NaN
2025-05-15,NaN,2.285071,NaN,0,0,NaN,NaN,NaN
2025-05-16,NaN,2.285071,NaN,0,0,NaN,NaN,NaN
2025-05-19,2.275563,2.285071,-0.416082,0,0,579.109985,50.138082,0.0
2025-05-20,NaN,2.285071,NaN,0,0,NaN,NaN,NaN
2025-05-21,NaN,2.285071,NaN,0,0,NaN,NaN,NaN
2025-05-22,NaN,2.285071,NaN,0,0,NaN,NaN,NaN
2025-05-23,NaN,2.285071,NaN,0,0,NaN,NaN,NaN


In [ ]:
def visualizing_buy_signal(buy_sell_signals,data):
    # create figure
    fig = make_subplots(rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.10,
        row_heights=[20,20,20],
        subplot_titles=("Buy signal","Sell signal","Close price and Buy/Sell moments"),
        specs = [[{}],[{}],[{"secondary_y": True}]]
    )

    # ROW 1 subplot: adding buy signal and decrease 

    fig.add_trace(
        go.Scatter(x=buy_sell_signals.index, y=buy_sell_signals['buy']*25-80, name="buy signal", mode='lines', line=dict(color='darkgreen'),connectgaps=True), 
        secondary_y=False, row=1,col=1)

    fig.add_trace(
        go.Scatter(x=buy_sell_signals.index, y=buy_sell_signals['buy signal decrease'], name="buy signal decrease from peaks", mode='lines', line=dict(color='green'),connectgaps=True), 
        secondary_y=False, row=1,col=1)
    
    fig.add_shape(type='line', x0=buy_sell_signals.index[0], x1=buy_sell_signals.index[-1], y0=-70, y1=-70, line=dict(dash='dash'), secondary_y=False, row=1, col=1)
    fig.add_shape(type='line', x0=buy_sell_signals.index[0], x1=buy_sell_signals.index[-1], y0=-80, y1=-80, line=dict(dash='dash'), secondary_y=False, row=1, col=1)

    fig.update_yaxes(range=[-120,50], row=1,col=2,secondary_y=False) 
    fig.update_yaxes(title_text="Percentage", row=1, col=1)

    # ROW 2 subplot: adding buy signal and decrease 
    fig.add_trace(
        go.Scatter(x=buy_sell_signals.index, y=buy_sell_signals['RSI_smoothed'], name="smoothed RSI signal", mode='lines', line=dict(color='red'),connectgaps=True), 
        secondary_y=False, row=2,col=1)
    
    fig.add_shape(type='line', x0=buy_sell_signals.index[0], x1=buy_sell_signals.index[-1], y0=65, y1=65, line=dict(dash='dash'), secondary_y=False, row=2, col=1)

    # ROW 3 subplot:
    fig.add_trace(
        go.Scatter(x=buy_sell_signals.index, y=buy_sell_signals['close'], name="Close price", line=dict(color='blue'),connectgaps=True), row=3, col=1,
        secondary_y=False)

    # Buy signals (green triangle up)
    buy_signals = buy_sell_signals[buy_sell_signals['buy light'] == 1]
    fig.add_trace(go.Scatter(
        x=buy_signals.index,
        y=buy_signals['close'],
        mode='markers',
        name='Buy Signal',
        marker=dict(symbol='triangle-up', color='lightgreen', size=10)
    ),row=3, col=1,secondary_y=False)

    buy_signals = buy_sell_signals[buy_sell_signals['buy strong'] == 2]
    fig.add_trace(go.Scatter(
        x=buy_signals.index,
        y=buy_signals['close'],
        mode='markers',
        name='Buy Signal',
        marker=dict(symbol='triangle-up', color='green', size=10)
    ),row=3, col=1,secondary_y=False)

    sell_signals = buy_sell_signals[buy_sell_signals['sell'] == 1]
    fig.add_trace(go.Scatter(
        x=sell_signals.index,
        y=sell_signals['close'],
        mode='markers',
        name='Sell Signal',
        marker=dict(symbol='triangle-down', color='red', size=10)
    ),row=3, col=1,secondary_y=False)

    fig.update_yaxes(title_text="Price", row=2, col=1)

    # Update layout
    fig.update_layout(
        width=1000,
        height=750,
        title="Buy Signal: Cut-off values",
        xaxis_title="Date",
    )

    fig.show()

visualizing_buy_signal(buy_sell_signals,indicator_total)

In [ ]:

def create_technical_chart():
    indicator_total, buy_sell_signals = computing_signals()
    
    fig = make_subplots(
        rows=6, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.4, 0.2, 0.2, 0.2,0.2,0.4],
        subplot_titles=("SPY Close Price", "RSI", "MACD", "Fear & Greed Index","VIX","Overall buy"),
        specs = [[{}],[{}],[{}],[{}],[{}],[{"secondary_y": True}]]
    )

    # Row 1: Close Price & 200DMA
    fig.add_trace(go.Scatter(x=indicator_total.index, y=indicator_total['Close'], name="Close", line=dict(color='blue'),connectgaps=True), row=1, col=1)
    fig.add_trace(go.Scatter(x=indicator_total.index, y=indicator_total['200DMA'], name="200DMA", line=dict(color='black'),connectgaps=True), row=1, col=1)
    fig.add_trace(go.Scatter(x=indicator_total.index, y=indicator_total['50DMA'], name="50DMA", line=dict(color='grey'),connectgaps=True), row=1, col=1)

    # Row 2: RSI
    fig.add_trace(go.Scatter(x=indicator_total.index, y=indicator_total['RSI'], name="RSI", line=dict(color='orange'),connectgaps=True), row=2, col=1)
    fig.add_trace(go.Scatter(x=indicator_total.index, y=indicator_total['RSI_Smoothed'], name="RSI", line=dict(color='coral'),connectgaps=True), row=2, col=1)

    fig.add_shape(type='line', x0=indicator_total.index[0], x1=indicator_total.index[-1], y0=65, y1=65, line=dict(dash='dash'), row=2, col=1)
    fig.add_shape(type='line', x0=indicator_total.index[0], x1=indicator_total.index[-1], y0=40, y1=40, line=dict(dash='dash'), row=2, col=1)

    # Row 3: MACD
    fig.add_trace(go.Scatter(x=indicator_total.index, y=indicator_total['MACD'], name="MACD", line=dict(color='green'),connectgaps=True), row=3, col=1)
    fig.add_trace(go.Scatter(x=indicator_total.index, y=indicator_total['MACD_signal'], name="Signal", line=dict(color='red'),connectgaps=True), row=3, col=1)
    fig.add_trace(go.Bar(x=indicator_total.index, y=indicator_total['MACD_hist'], name="Histogram", marker_color='gray'), row=3, col=1)

    # Row 4: FGI (if available)
    if indicator_total['FGI'] is not None:
        fig.add_trace(go.Scatter(x=indicator_total.index, y=indicator_total['FGI'], name="FGI", line=dict(color='purple'),connectgaps=True), row=4, col=1)
    fig.add_shape(type='line', x0=indicator_total.index[0], x1=indicator_total.index[-1], y0=70, y1=70, line=dict(dash='dash'), row=4, col=1)
    fig.add_shape(type='line', x0=indicator_total.index[0], x1=indicator_total.index[-1], y0=30, y1=30, line=dict(dash='dash'), row=4, col=1)
    
    # Row 5: VIX 
    fig.add_trace(go.Scatter(x=indicator_total.index, y=indicator_total['^VIX'], name="VIX", line=dict(color='yellow'),connectgaps=True), row=5, col=1)
    fig.add_shape(type='line', x0=indicator_total.index[0], x1=indicator_total.index[-1], y0=15, y1=15, line=dict(dash='dash'), row=5, col=1)

    # Row 6: buy and sell markers
    fig.add_trace(go.Scatter(x=buy_sell_signals.index, y=buy_sell_signals['close'], name="Close", line=dict(color='blue'),connectgaps=True), row=6, col=1,secondary_y=False)

    # Sell signals (red triangle down)
    buy_signals = buy_sell_signals[buy_sell_signals['buy light'] == 1]
    fig.add_trace(go.Scatter(
        x=buy_signals.index,
        y=buy_signals['close'],
        mode='markers',
        name='Buy Signal',
        marker=dict(symbol='triangle-up', color='lightgreen', size=10)
    ),row=6, col=1,secondary_y=False)

    buy_signals = buy_sell_signals[buy_sell_signals['buy strong'] == 2]
    fig.add_trace(go.Scatter(
        x=buy_signals.index,
        y=buy_signals['close'],
        mode='markers',
        name='Buy Signal',
        marker=dict(symbol='triangle-up', color='green', size=10)
    ),row=6, col=1,secondary_y=False)

    sell_signals = buy_sell_signals[buy_sell_signals['sell'] == 1]
    fig.add_trace(go.Scatter(
        x=sell_signals.index,
        y=sell_signals['close'],
        mode='markers',
        name='Sell Signal',
        marker=dict(symbol='triangle-down', color='red', size=10)
    ),row=6, col=1,secondary_y=False)

    fig.update_yaxes(title_text="Price", row=2, col=1)

    fig.update_layout(
        height=1100,
        title_text="SPY Technical Indicators",
        showlegend=True
    )
    return fig

create_technical_chart()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [124]:
def evaluate(buy_sell_signals):    
    
    # determining buy and sell signal dates
    buy_med = buy_sell_signals[buy_sell_signals['buy light']==1].index
    buy_strong = buy_sell_signals[buy_sell_signals['buy strong']==2].index
    sell = buy_sell_signals[buy_sell_signals['sell']==1].index

    # start parameters
    spendible_cap = 14000
    start_cap = 14000
    start_date = '2022-01-31'
    shares_bought = 0
    spend = 0
    profit = 0
    sold = 0
    
    shares_boughts = []
    worths = []
    profits = []
    spends = []
    spendible_caps = []
    solds=[]

    # STRATEGY II
    shares_bought_II = spendible_cap / buy_sell_signals.loc[start_date,'close']
    shares_bought_II = shares_bought_II* np.ones(len(buy_sell_signals.loc[start_date:]))
    profit_II = (buy_sell_signals.loc[start_date:,'close'] * shares_bought_II) - spendible_cap

    for index, rows in buy_sell_signals.iterrows():
        
        # per date if buy or sell
        if index > pd.to_datetime(start_date):

            if spendible_cap > 0:    
                if index in buy_med:
                    bought = 0.07 * start_cap
                    shares_bought += bought/rows['close'] 
                    spendible_cap = spendible_cap - bought
                    spend += bought
                    spends.append(spend)

                if index in buy_strong:
                    bought = 0.14 * start_cap
                    shares_bought += bought/rows['close'] 
                    spendible_cap = spendible_cap - bought
                    spend += bought
                    spends.append(spend)

            if index in sell:
                shares_bought -= (0.5*last_worth)/rows['close']
                spendible_cap = spendible_cap + 0.5*last_worth
                sold += 0.5*last_worth
                solds.append(sold)

                start_cap = spendible_cap
            else:
                sold = sold

        if shares_bought is not 0:
            worth = shares_bought*rows['close'] + sold
            worths.append(worth)
            
            profit = worth - spend
            profits.append(profit)
            spendible_caps.append(spendible_cap)
            shares_boughts.append(shares_bought)
            if not np.isnan(worth):
                last_worth = worth
        else:
            worths.append(0)
            profits.append(0)
            spendible_caps.append(14000)
            shares_boughts.append(0)


    ## figure
    fig = make_subplots(
        rows=2,cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        subplot_titles=('Worth & Profits','Shares Bought')
    )

    # figure ROW 1
    fig.add_trace(go.Scatter(
        x=buy_sell_signals.index, y=worths,name='worth',line=dict(color='blue'),connectgaps=True),row=1,col=1)
    fig.add_trace(go.Scatter(
        x=buy_sell_signals.index, y=profits,name='profit my strategy',line=dict(color='orange'),connectgaps=True),row=1,col=1)
    fig.add_trace(go.Scatter(
        x=buy_sell_signals.loc[start_date:].index, y=profit_II,name='proft strategy HOLD',line=dict(color='red'),connectgaps=True),row=1,col=1)
    fig.add_trace(go.Scatter(
        x=buy_sell_signals.index, y=spendible_caps,name='spendible capital',line=dict(color='green'),connectgaps=True),row=1,col=1)
    
    fig.update_layout(
    height=700,
    width=1300,
    title_text="Evaluation",
    showlegend=True
    )

    # figure ROW 2
    fig.add_trace(go.Scatter(
        x=buy_sell_signals.index, y=shares_boughts,name='shares bought my strategy',line=dict(color='blue'),connectgaps=True),row=2,col=1)
    fig.add_trace(go.Scatter(
        x=buy_sell_signals.loc[start_date:].index, y=shares_bought_II,name='shares bought strategy HOLD',line=dict(color='red'),connectgaps=True),row=2,col=1)

    fig.show()

    # result
    profit_print = [x for x in profits if not np.isnan(x)]
    print(f'THE WINST OP {buy_sell_signals.index[-1]} met een winst van MY STRATEGY: {profit_print[-1]}')

    profit_II_print = [x for x in profit_II if not np.isnan(x)]
    print(f'THE WINST OP {buy_sell_signals.index[-1]} met een winst van HOLD STRATEGY: {profit_II_print[-1]}')

evaluate(buy_sell_signals)

<>:59: SyntaxWarning:

"is not" with a literal. Did you mean "!="?

<>:59: SyntaxWarning:

"is not" with a literal. Did you mean "!="?

C:\Users\bwava\AppData\Local\Temp\ipykernel_26120\2469344725.py:59: SyntaxWarning:

"is not" with a literal. Did you mean "!="?



THE WINST OP 2025-06-06 00:00:00 met een winst van MY STRATEGY: 9043.25609314998
THE WINST OP 2025-06-06 00:00:00 met een winst van HOLD STRATEGY: 5592.068616264751
